In [15]:
%matplotlib ipympl

import os
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from scipy import stats
import gsw

In [2]:
ds = xr.open_dataset('data/MBARI_StationM_ADCP_201711_201811.nc')

In [3]:
ds

<xarray.Dataset>
Dimensions:    (beam: 4, bin: 23, time: 98208)
Coordinates:
  * time       (time) datetime64[ns] 2017-11-10T10:03:13.760000 ... 2018-10-17T09:58:13.720000
Dimensions without coordinates: beam, bin
Data variables:
    Pit        (time) float64 ...
    Rol        (time) float64 ...
    Hea        (time) float64 ...
    Tem        (time) float64 ...
    Dep        (time) float64 ...
    Ori        (time) int64 ...
    BIT        (time) int64 ...
    Bat        (time) int64 ...
    Eas        (time, bin) float64 ...
    Nor        (time, bin) float64 ...
    Ver        (time, bin) float64 ...
    Err        (time, bin) float64 ...
    EA         (time, bin, beam) float64 ...
    PG         (time, bin, beam) float64 ...
    C          (time, bin, beam) float64 ...
    binheight  (bin) float64 ...
    Eas_tide   (time, bin) float64 ...
    Nor_tide   (time, bin) float64 ...
    Eas_filt   (time, bin) float64 ...
    Nor_filt   (time, bin) float64 ...
Attributes:
    PingsPer

In [4]:
bini = np.arange(1,17)

plt.figure()
plt.plot(np.mean(ds['Eas'][:,bini],axis=0),ds['binheight'][bini],'-o')
plt.plot(np.mean(ds['Nor'][:,bini],axis=0),ds['binheight'][bini],'-o')
plt.title('mean velocity - entire deployment')
plt.legend(['East','North'])
plt.ylabel('height above instrument')
plt.xlabel('[m/s]')

FigureCanvasNbAgg()

Text(0.5, 0, '[m/s]')

In [5]:
dshr = ds.resample(time='1H').mean()

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/xarray/core/nanops.py:161: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [6]:
dshr

<xarray.Dataset>
Dimensions:    (beam: 4, bin: 23, time: 8184)
Coordinates:
  * time       (time) datetime64[ns] 2017-11-10T10:00:00 ... 2018-10-17T09:00:00
Dimensions without coordinates: beam, bin
Data variables:
    Pit        (time) float64 6.938 6.96 6.971 6.981 ... 7.184 7.184 7.184 7.183
    Rol        (time) float64 -0.1008 -0.1167 -0.1217 ... -0.2292 -0.23 -0.2283
    Hea        (time) float64 277.3 277.3 277.3 277.3 ... 277.9 277.9 277.9
    Tem        (time) float64 1.421 1.415 1.41 1.41 ... 1.412 1.413 1.413 1.414
    Dep        (time) float64 4e+03 4e+03 4e+03 4e+03 ... 4e+03 4e+03 4e+03
    Ori        (time) float64 1.0 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0 1.0
    BIT        (time) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Bat        (time) float64 116.8 116.1 116.0 116.0 ... 114.0 114.0 114.0
    Eas        (time, bin) float64 0.01917 0.02317 0.02483 ... -0.038 nan
    Nor        (time, bin) float64 0.0009167 0.00575 0.005 ... 0.01221 nan
    Ver    

In [99]:
nbins = 7
zi = np.arange(0,nbins)

In [100]:
plt.figure()
plt.subplot(211)
plt.plot(dshr.time,dshr.Eas[:,zi]);

plt.subplot(212)
plt.plot(dshr.time,dshr.Nor[:,zi]);

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [101]:
N = len(dshr.time)
slopeu = np.nan*np.ones(N)
interceptu = np.nan*np.ones(N)
ru = np.nan*np.ones(N)
pu = np.nan*np.ones(N)
slopev = np.nan*np.ones(N)
interceptv = np.nan*np.ones(N)
rv = np.nan*np.ones(N)
pv = np.nan*np.ones(N)

for ti in np.arange(N):
    resultu = stats.linregress(np.log(dshr.binheight[ti,zi]+0.4572),dshr.Eas[ti,zi])
    resultv = stats.linregress(np.log(dshr.binheight[ti,zi]+0.4572),dshr.Nor[ti,zi])
    slopeu[ti] = resultu[0]
    interceptu[ti] = resultu[1]
    ru[ti] = resultu[2]
    pu[ti] = resultu[3]
    slopev[ti] = resultv[0]
    interceptv[ti] = resultv[1]
    rv[ti] = resultv[2]
    pv[ti] = resultv[3]

In [102]:
plt.figure()
plt.plot(dshr.Eas[:,0],ru,'.')

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [103]:
pthresh = 0.01

In [104]:
kappa = 0.41
ustaru = kappa*slopeu
zou = np.exp(-kappa*interceptu/ustaru)

ustarv = kappa*slopev
zov = np.exp(-kappa*interceptv/ustarv)

piu, = np.where((pu < pthresh) & (zou < 100))
piv, = np.where((pv < pthresh) & (zov < 100))

In [105]:
plt.figure()
plt.plot(dshr.time[piu],np.log(zou[piu]),'.')

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [106]:
plt.figure()
plt.hist(np.log(zou[piu]),20);

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [107]:
np.exp(np.mean(np.log(zou[piu])))

0.16222167402795765

In [108]:
np.exp(np.mean(np.log(zov[piv])))

0.24963982688886793

In [109]:
plt.figure()
plt.plot(dshr.Eas[piu,0],np.log(zou[piu]),'.')
plt.plot(dshr.Nor[piv,0],np.log(zov[piv]),'.')
plt.xlabel('u or v [m/s]')
plt.ylabel('ln($z_o$)')
plt.legend(['u','v'])

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [114]:
plt.figure()
plt.scatter(dshr.Eas[piu,0],np.log(zou[piu]),8,ru[piu])
plt.xlabel('u [m/s]')
plt.ylabel('ln($z_o$)')

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

Text(0, 0.5, 'ln($z_o$)')